In [31]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor = torch.randn(2, 2).to(device)
print(tensor.device)

print(torch.cuda.device_count())  # Imprime el número de GPUs disponibles

cuda:0
1


In [32]:
!nvidia-smi

Sun Oct 27 13:21:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8             10W /  115W |     109MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install elementpath
!pip install tqdm
!pip install ultralytics

In [7]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
from pathlib import Path

In [11]:
import yaml
from pathlib import Path
import requests
import zipfile

def download(urls, dir, exist_ok=True):
    dir.mkdir(parents=True, exist_ok=True)  # Crea el directorio si no existe
    for url in urls:
        filename = url.split('/')[-1]
        file_path = dir / filename

        # Descargar el archivo si no existe o si exist_ok=False
        if not file_path.exists() or not exist_ok:
            print(f"Descargando {filename}...")
            response = requests.get(url, stream=True)
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

        # Extraer el archivo si es un .zip
        if file_path.suffix == '.zip':
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(dir)
            print(f"Descomprimido {filename}")

In [14]:
import yaml
from pathlib import Path

# Cargar el archivo VOC.yaml con codificación UTF-8
with open(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOC.yaml', encoding='utf-8') as f:
    yaml_data = yaml.safe_load(f)

# Configurar la ruta raíz para guardar en "content" de windows
dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2')

# Definir URLs de los archivos para descargar
url = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'
urls = [
    f'{url}VOCtrainval_06-Nov-2007.zip',  # 446MB, 5012 imágenes
    f'{url}VOCtest_06-Nov-2007.zip',      # 438MB, 4953 imágenes
    f'{url}VOCtrainval_11-May-2012.zip'   # 1.95GB, 17126 imágenes
]

# Llamar a la función download para descargar y descomprimir en el directorio especificado
download(urls, dir=dir, exist_ok=True)


Descargando VOCtrainval_06-Nov-2007.zip...
Descomprimido VOCtrainval_06-Nov-2007.zip
Descargando VOCtest_06-Nov-2007.zip...
Descomprimido VOCtest_06-Nov-2007.zip
Descargando VOCtrainval_11-May-2012.zip...
Descomprimido VOCtrainval_11-May-2012.zip


In [18]:
from tqdm import tqdm
import yaml
from pathlib import Path
import xml.etree.ElementTree as ET
import os
import shutil

def convert_label(path, lb_path, year, image_id, yaml_data):
    try:
        # Verificar que el archivo XML existe
        xml_path = path / f'VOC{year}/Annotations/{image_id}.xml'
        if not xml_path.exists():
            print(f"XML no encontrado: {xml_path}")
            return False

        in_file = open(xml_path)
        out_file = open(lb_path, 'w')
        tree = ET.parse(in_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        # Verificar que el archivo YAML contiene las clases correctas
        names = list(yaml_data['names'].values())

        # Contador para verificar si se escriben etiquetas
        labels_written = 0

        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls in names and int(obj.find('difficult').text) != 1:
                xmlbox = obj.find('bndbox')
                xmin = float(xmlbox.find('xmin').text)
                xmax = float(xmlbox.find('xmax').text)
                ymin = float(xmlbox.find('ymin').text)
                ymax = float(xmlbox.find('ymax').text)

                # Verificar que las coordenadas son válidas
                if xmin >= xmax or ymin >= ymax:
                    print(f"Coordenadas inválidas en {image_id}: xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}")
                    continue

                # Convertir coordenadas
                dw = 1./w
                dh = 1./h
                x = (xmin + xmax)/2.0 - 1
                y = (ymin + ymax)/2.0 - 1
                w_box = xmax - xmin
                h_box = ymax - ymin

                x = x*dw
                w_box = w_box*dw
                y = y*dh
                h_box = h_box*dh

                cls_id = names.index(cls)
                out_file.write(f"{cls_id} {x:.6f} {y:.6f} {w_box:.6f} {h_box:.6f}\n")
                labels_written += 1

        in_file.close()
        out_file.close()

        # Verificar si se escribieron etiquetas
        if labels_written == 0:
            print(f"No se escribieron etiquetas para {image_id}")
            if os.path.exists(lb_path):
                os.remove(lb_path)
            return False

        return True

    except Exception as e:
        print(f"Error procesando {image_id}: {str(e)}")
        return False

# Cargar el archivo YAML
with open(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOC.yaml',  encoding='utf-8') as f:
    yaml_data = yaml.safe_load(f)
    print("Clases disponibles:", yaml_data['names'])

# Establecer la ruta del dataset raíz
dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit')

# Contador para estadísticas
stats = {'procesados': 0, 'exitosos': 0, 'fallidos': 0}

for year, image_set in ('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test'):
    print(f"\nProcesando {year}/{image_set}")

    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    list_file = dir / f'VOC{year}/ImageSets/Main/{image_set}.txt'

    if not list_file.exists():
        print(f"Archivo de lista no encontrado: {list_file}")
        continue

    with open(list_file) as f:
        image_ids = f.read().strip().split()

    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        stats['procesados'] += 1

        img_file = dir / f'VOC{year}/JPEGImages/{id}.jpg'
        lb_path = (lbs_path / img_file.name).with_suffix('.txt')

        if img_file.exists():
            # Copiar en lugar de mover la imagen
            shutil.copy2(img_file, imgs_path / img_file.name)

            if convert_label(dir, lb_path, year, id, yaml_data):
                stats['exitosos'] += 1
            else:
                stats['fallidos'] += 1
        else:
            print(f"Imagen no encontrada: {img_file}")
            stats['fallidos'] += 1

# Imprimir estadísticas finales
print("\nEstadísticas de conversión:")
print(f"Total procesados: {stats['procesados']}")
print(f"Convertidos exitosamente: {stats['exitosos']}")
print(f"Fallidos: {stats['fallidos']}")

# Verificar algunas etiquetas generadas
print("\nVerificando algunas etiquetas generadas:")
for year, image_set in ('2012', 'train'), ('2012', 'val'):
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    if lbs_path.exists():
        label_files = list(lbs_path.glob('*.txt'))
        if label_files:
            sample_file = label_files[0]
            print(f"\nContenido de {sample_file}:")
            with open(sample_file, 'r') as f:
                print(f.read())

Clases disponibles: {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}

Procesando 2012/train


train2012: 100%|██████████| 5717/5717 [00:54<00:00, 104.10it/s]



Procesando 2012/val


val2012: 100%|██████████| 5823/5823 [01:00<00:00, 96.78it/s] 



Procesando 2007/train


...


Procesando 2007/val


val2007: 100%|██████████| 2510/2510 [00:18<00:00, 137.79it/s]



Procesando 2007/test


test2007: 100%|██████████| 4952/4952 [00:48<00:00, 102.78it/s]



Estadísticas de conversión:
Total procesados: 21503
Convertidos exitosamente: 21503
Fallidos: 0

Verificando algunas etiquetas generadas:

Contenido de C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\train2012\2008_000008.txt:
12 0.522000 0.571267 0.836000 0.753394
14 0.445000 0.236425 0.262000 0.278281


Contenido de C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\val2012\2008_000002.txt:
19 0.480000 0.402667 0.828000 0.752000



In [19]:
 import os

# ruta de la carpeta
dir_path = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\train2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5717


In [20]:
 import os

# ruta de la carpeta
dir_path = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\val2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5823


In [21]:
 import os

# ruta de la carpeta
dir_path = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\train2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5717


In [23]:
 import os

# ruta de la carpeta
dir_path = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\val2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5823


In [24]:
import os

# Directorios de imágenes y etiquetas
train_images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\train2012'
train_labels_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\train2012'
val_images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\val2012'
val_labels_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\val2012'

# Función para obtener la lista de archivos sin la extensión
def obtener_archivos_sin_extension(directorio, extension):
    archivos = [os.path.splitext(f)[0] for f in os.listdir(directorio) if f.endswith(extension)]
    return set(archivos)

# Obtener los archivos de imágenes y etiquetas (sin extensión)
train_images = obtener_archivos_sin_extension(train_images_dir, '.jpg')
train_labels = obtener_archivos_sin_extension(train_labels_dir, '.txt')
val_images = obtener_archivos_sin_extension(val_images_dir, '.jpg')
val_labels = obtener_archivos_sin_extension(val_labels_dir, '.txt')

# Comparar imágenes y etiquetas en train y val
imagenes_sin_etiquetas_train = train_images - train_labels
imagenes_sin_etiquetas_val = val_images - val_labels

# Mostrar imágenes sin etiquetas sin eliminarlas
print(f'Imágenes sin etiquetas en train: {len(imagenes_sin_etiquetas_train)}')
print(f'Imágenes sin etiquetas en val: {len(imagenes_sin_etiquetas_val)}')

# Eliminar imágenes sin etiquetas en train
if imagenes_sin_etiquetas_train:
    print('Eliminando imágenes sin etiquetas en train:')
    for img in imagenes_sin_etiquetas_train:
        img_path = os.path.join(train_images_dir, img + '.jpg')
        print(f'Eliminando {img_path}')
        os.remove(img_path)  # Eliminar la imagen

# Eliminar imágenes sin etiquetas en val
if imagenes_sin_etiquetas_val:
    print('Eliminando imágenes sin etiquetas en val:')
    for img in imagenes_sin_etiquetas_val:
        img_path = os.path.join(val_images_dir, img + '.jpg')
        print(f'Eliminando {img_path}')
        os.remove(img_path)  # Eliminar la imagen

Imágenes sin etiquetas en train: 0
Imágenes sin etiquetas en val: 0


In [26]:
import os
import random
import shutil
from pathlib import Path


val_images_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\val2012')
val_labels_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\val2012')
new_val_images_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\images\val2012_reduced')
new_val_labels_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\VOCdevkit\labels\val2012_reduced')

# Crear directorios para las nuevas imágenes y etiquetas si no existen
new_val_images_dir.mkdir(exist_ok=True, parents=True)
new_val_labels_dir.mkdir(exist_ok=True, parents=True)

# Obtener la lista de imágenes y etiquetas en la carpeta de validación
val_images = list(val_images_dir.glob('*.jpg'))  # Asumiendo que las imágenes son .jpg
val_labels = [val_labels_dir / (img.stem + '.txt') for img in val_images]

# Comprobar que la cantidad de imágenes y etiquetas coincidan
assert len(val_images) == len(val_labels), "La cantidad de imágenes y etiquetas no coincide."

# Seleccionar aleatoriamente 1200 imágenes y etiquetas
selected_indices = random.sample(range(len(val_images)), 1200)

# Mover las imágenes y etiquetas seleccionadas a la nueva carpeta
for idx in selected_indices:
    shutil.move(str(val_images[idx]), str(new_val_images_dir / val_images[idx].name))
    shutil.move(str(val_labels[idx]), str(new_val_labels_dir / val_labels[idx].name))

print(f"Se han seleccionado y movido {len(selected_indices)} imágenes y etiquetas al nuevo conjunto de validación.")


Se han seleccionado y movido 1200 imágenes y etiquetas al nuevo conjunto de validación.


In [29]:
import os

# Directorio de las etiquetas
images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\dataset\images\train'

# Directorio donde se guardará el archivo 2017.txt
output_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\dataset'

# Nombre del archivo de salida
output_file = 'train.txt'

# Función para obtener las rutas de las imágenes
def get_image_paths(label_dir):
    image_paths = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.jpg'):
            # Asumimos que el nombre del archivo de imagen es el mismo que el de la etiqueta, pero con extensión .jpg
            image_name = os.path.splitext(label_file)[0] + '.jpg'
            # Construimos la ruta relativa de la imagen
            image_path = f'./images/train/{image_name}'
            image_paths.append(image_path)
    return image_paths

# Obtener las rutas de las imágenes
image_paths = get_image_paths(images_dir)

# Escribir las rutas en el archivo 2017.txt
output_path = os.path.join(output_dir, output_file)
with open(output_path, 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

print(f"Se ha creado el archivo {output_file} con {len(image_paths)} rutas de imágenes.")

Se ha creado el archivo train.txt con 5717 rutas de imágenes.


In [30]:
import os

# Directorio de las etiquetas
images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\dataset\images\val'

# Directorio donde se guardará el archivo 2017.txt
output_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\dataset'


# Nombre del archivo de salida
output_file = 'val.txt'

# Función para obtener las rutas de las imágenes
def get_image_paths(label_dir):
    image_paths = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.jpg'):
            # Asumimos que el nombre del archivo de imagen es el mismo que el de la etiqueta, pero con extensión .jpg
            image_name = os.path.splitext(label_file)[0] + '.jpg'
            # Construimos la ruta relativa de la imagen
            image_path = f'./images/val/{image_name}'
            image_paths.append(image_path)
    return image_paths

# Obtener las rutas de las imágenes
image_paths = get_image_paths(images_dir)

# Escribir las rutas en el archivo 2017.txt
output_path = os.path.join(output_dir, output_file)
with open(output_path, 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

print(f"Se ha creado el archivo {output_file} con {len(image_paths)} rutas de imágenes.")

Se ha creado el archivo val.txt con 1200 rutas de imágenes.


In [34]:
# 4. Entrenar el modelo YOLOv8

!pip install ultralytics
from ultralytics import YOLO

# Cargar el modelo YOLOv8n para clasificación
model = YOLO('yolo11s.pt')
# Entrenar el modelo con CIFAR-10
model.train(data='VOC_FINAL.yaml', epochs=20, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.23 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=VOC_FINAL.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/luisangelolveraolvera/general/def49727043f413c8ce0ee84d7690eb5

COMET INFO: Couldn't find a Git repository in 'C:\\Users\\User\\Documents\\JUPYTER_LOCAL\\IMAGENES_ENTRENAMIENTO\\DETECCION\\PASCALV2' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...


100%|██████████| 5.35M/5.35M [00:00<00:00, 10.2MB/s]


AMP: checks passed 


...


20 epochs completed in 0.585 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 19.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 19.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11s summary (fused): 238 layers, 9,420,540 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:21<00:00,  1.74it/s]


                   all       1200       2819      0.795      0.738      0.796      0.618
             aeroplane         59         67      0.903      0.851        0.9      0.753
               bicycle         65         84      0.826      0.738      0.849      0.665
                  bird         88        126      0.773      0.698      0.759       0.57
                  boat         52         74      0.734      0.662      0.704      0.474
                bottle         68        119      0.763      0.595      0.702      0.453
                   bus         50         77      0.883      0.783      0.911      0.772
                   car        115        229      0.871      0.734      0.838       0.63
                   cat        132        151      0.839      0.834      0.885      0.731
                 chair        122        269      0.766      0.596      0.714      0.515
                   cow         31         59       0.86      0.864      0.861      0.709
           diningtabl

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : stuck_blend_870
COMET INFO:     url                   : https://www.comet.com/luisangelolveraolvera/general/def49727043f413c8ce0ee84d7690eb5
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [21]               : (2.4811500000000018e-05, 0.0003753671703910615)
COMET INFO:     lr/pg1 [21]               : (2.4811500000000018e-05, 0.0003753671703910615)
COMET INFO:     lr/pg2 [21]               : (2.4811500000000018e-05, 0.0003753671703910615)
COMET INFO:     metrics/mAP50(B) [21]     : (0.59944, 0.79628)
COMET INFO:     metrics/mAP50-95(B) [21]  : (0.39733, 0.61812)
COMET INFO:     metrics/precision(B) [21] : (0.62082, 0.79941)
COMET INFO

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000187128467B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0

In [37]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Ruta del directorio donde se encuentran las imágenes
directory = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\DETECCION\PASCALV2\runs\detect\train2'

# Listar todos los archivos en el directorio
files = os.listdir(directory)
print("Estos son los archivos de resultado de entrenamiento", files)

# Filtrar solo los archivos de imagen (extensiones comunes)
image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg'))]

# Mostrar las imágenes con control para evitar demasiadas figuras abiertas
if image_files:
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)

        # Cargar la imagen
        image = Image.open(image_path)

        # Mostrar la imagen
        plt.figure()
        plt.imshow(image)
        plt.axis('off')  # Ocultar ejes
        plt.show()

        # Cerrar la figura para liberar memoria
        plt.close()


Estos son los archivos de resultado de entrenamiento ['args.yaml', 'confusion_matrix.png', 'confusion_matrix_normalized.png', 'F1_curve.png', 'labels.jpg', 'labels_correlogram.jpg', 'PR_curve.png', 'P_curve.png', 'results.csv', 'results.png', 'R_curve.png', 'train_batch0.jpg', 'train_batch1.jpg', 'train_batch2.jpg', 'train_batch3580.jpg', 'train_batch3581.jpg', 'train_batch3582.jpg', 'val_batch0_labels.jpg', 'val_batch0_pred.jpg', 'val_batch1_labels.jpg', 'val_batch1_pred.jpg', 'val_batch2_labels.jpg', 'val_batch2_pred.jpg', 'weights']


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>